In [1]:
import os
import re
import pandas as pd

import requests
from bs4 import BeautifulSoup
import csv

In [2]:
BASE_URL = "https://www.consiglio.vda.it/app/oggettidelconsiglio/dettaglio?pk_documento={}&versione=R"

OUTPUT_FOLDER = "/home/sylcherry/UNIVDA/downloads"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

#funzione per scraperare e salvare i resoconti dal sito del consiglio Valle
def scrape_and_save(doc_id):
    """
    Scarica e salva il contenuto della pagina come file di testo.
    """
    url = BASE_URL.format(doc_id)
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Documento {doc_id} non trovato (HTTP {response.status_code}).")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    page_text = soup.get_text(separator="\n", strip=True)

    output_file = os.path.join(OUTPUT_FOLDER, f"{doc_id}.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(page_text)
    
    print(f"Documento {doc_id} salvato in: {output_file}")
    return output_file

def main(start_id, end_id):
    """
    Itera sugli ID dei documenti, scarica le pagine e le salva come file di testo.
    Genera un file CSV con i percorsi salvati.
    """
    csv_data = []

    print(f"Inizio lo scraping per i documenti dal {start_id} al {end_id}...")
    for doc_id in range(start_id, end_id + 1):
        print(f"Processo il documento {doc_id}...")
        file_path = scrape_and_save(doc_id)
        if file_path:
            csv_data.append({"ID_file": doc_id, "path_src": file_path})

    csv_file = "csv_paths.csv"
    print(f"Salvataggio dei risultati nel file CSV: {csv_file}")
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["ID_file", "path_src"])
        writer.writeheader()
        writer.writerows(csv_data)

    print("Completato! Tutti i documenti salvati nella cartella:", OUTPUT_FOLDER)

if __name__ == "__main__":
    main(start_id=47970, end_id=47990)

Inizio lo scraping per i documenti dal 47970 al 47990...
Processo il documento 47970...
Documento 47970 salvato in: /home/sylcherry/UNIVDA/downloads/47970.txt
Processo il documento 47971...
Documento 47971 salvato in: /home/sylcherry/UNIVDA/downloads/47971.txt
Processo il documento 47972...
Documento 47972 non trovato (HTTP 404).
Processo il documento 47973...
Documento 47973 non trovato (HTTP 404).
Processo il documento 47974...
Documento 47974 non trovato (HTTP 404).
Processo il documento 47975...
Documento 47975 non trovato (HTTP 404).
Processo il documento 47976...
Documento 47976 non trovato (HTTP 404).
Processo il documento 47977...
Documento 47977 non trovato (HTTP 404).
Processo il documento 47978...
Documento 47978 non trovato (HTTP 404).
Processo il documento 47979...
Documento 47979 non trovato (HTTP 404).
Processo il documento 47980...
Documento 47980 non trovato (HTTP 404).
Processo il documento 47981...
Documento 47981 salvato in: /home/sylcherry/UNIVDA/downloads/47981.tx

In [3]:
#funzione per pulire i file txt dalle parti inutili presenti nella pagina web del consiglio Valle
def first_clean(target_folder, keyword, mid_strings, end_keyword, csv_paths):
    df = pd.read_csv(csv_paths)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    df["path_clean"] = ""

    for idx, row in df.iterrows():
        source_path = row["path_src"]
        if source_path.endswith(".txt"):
            filename = os.path.basename(source_path)
            target_path = os.path.join(target_folder, filename)

            df.at[idx, "path_clean"] = target_path

            with open(source_path, "r", encoding="utf-8") as file:
                content = file.read()

            if keyword in content:
                content = keyword + content.split(keyword, 1)[1] 

            for mid_string in mid_strings:
                content = re.sub(re.escape(mid_string), "", content) 

            if end_keyword in content:
                content = content.split(end_keyword, 1)[0]  

            with open(target_path, "w", encoding="utf-8") as file:
                file.write(content)

    df.to_csv(csv_paths, index=False)

#funzione per estrarre informazioni dai file txt: oggetto, legislatura e classificazione
def first_info(pattern1, pattern2, csv_paths, csv_details):
    df_paths = pd.read_csv(csv_paths)
    df_details = pd.DataFrame({'ID_file': df_paths['ID_file']})

    df_details['object'] = ""
    df_details['legislature'] = ""
    df_details['class'] = ""

    for idx, row in df_paths.iterrows():
        source_path = row["path_clean"]
        if source_path.endswith(".txt"):
            filename = os.path.basename(source_path)
            filenum = int(os.path.splitext(filename)[0])

            with open(source_path, "r", encoding="utf-8") as file:
                content = file.read()

            match1 = re.search(pattern1, content)

            if match1:
                stringa = match1.group(1).strip()
                ogg, leg = stringa.split("/", 1)

                df_details.loc[df_details['ID_file'] == filenum, 'object'] = ogg
                df_details.loc[df_details['ID_file'] == filenum, 'legislature'] = leg

            match2 = re.search(pattern2, content, re.DOTALL)
            
            if match2:
                classe = match2.group(1).strip()
                classe = classe.replace("\n", ", ").strip()
                df_details.loc[df_details['ID_file'] == filenum, 'class'] = classe

    df_details.to_csv(csv_details, index=False)

In [4]:
target_folder = "/home/sylcherry/UNIVDA/clean"

csv_paths = "/home/sylcherry/UNIVDA/csv_paths.csv"
csv_details = "/home/sylcherry/UNIVDA/csv_details.csv"

In [5]:
keyword = "Classificazione"
mid_strings = ["Precedente", "Successivo", "Resoconto integrale del dibattito dell'aula. I documenti allegati sono reperibili nel link \"iter atto\"."]
end_keyword = "Informativa cookies"

first_clean(target_folder, keyword, mid_strings, end_keyword, csv_paths)

In [6]:
pattern1 = r'(?:OGGETTO N\.|OBJET N°)(.*?)\s*-'
pattern2 = r'Classificazione\s*(.*?)\s*Oggetto'

first_info(pattern1, pattern2, csv_paths, csv_details)